In [2]:
import pandas as pd
import numpy as np
import os
import re
from scipy.io.wavfile import read
import wave
from tqdm import tqdm

In [ ]:
def is_wav(f):
    res = True
    try:
        wave.open(f)
    except wave.Error as e:
        res = False
    return res

def pcm2wav(pcm_file, save_file, channels=1, bits=16, sample_rate=16000):

    if is_wav(pcm_file):
        raise ValueError('"' + str(pcm_file) + '"' +
                         " is a wav file, not pcm file! ")

    pcmf = open(pcm_file, 'rb')
    pcmdata = pcmf.read()
    pcmf.close()

    if bits % 8 != 0:
        raise ValueError("bits % 8 must == 0. now bits:" + str(bits))

    wavfile = wave.open(save_file, 'wb')

    wavfile.setnchannels(channels)
    wavfile.setsampwidth(bits // 8)
    wavfile.setframerate(sample_rate)

    wavfile.writeframes(pcmdata)
    wavfile.close()

# 1. hackathon data

In [43]:
hacka_dir = '/Data/etc/Robust_ASR/data/hacka/'
hacka_label = '000.PCM2TEXT'
hacka_cats = os.listdir(hacka_dir)
hacka_cats.remove('.ipynb_checkpoints')

In [11]:
path_list = []
label_list = []
for category in hacka_cats:
    hacka_label_path = os.listdir(os.path.join(hacka_dir, category, hacka_label))[0]
    with open(os.path.join(hacka_dir, category, hacka_label, hacka_label_path), 'r', encoding='cp949')as f:
        while True:
            row = f.readline()
            if not row:
                break
            row = row.replace('\\','/').replace('../2020_자유대화_Hackarthon_학습DB/',hacka_dir )
            path, label = row.split('\t')
            path_list.append(path)
            label_list.append(label)

In [120]:
sym_including = list(filter(lambda x : re.compile('\(\w.*\)').search(x), label_list))

In [121]:
sym_including

['술은 술은 안 먹어야지 (SP:그르) 그나마 쬐금 도움이 되는 거 같애\n',
 '그렇다고 건강관리 안 했다고 해서 (SP:안 아) 안 아픈 것도 아니고\n',
 '이게 (SP:사제) 사계절이 있는 우리나라가 참 내가 사는 곳이지만\n',
 '내가 너한테 얘기 했었나 우리 아들 며칠 전에 베트남 갔다 (SP:왔가) 왔다고\n',
 '(FP:아) 런던\n',
 '(FP:어)레몬\n',
 '(FP:어)룸메이트\n',
 '(FP:어)리더\n',
 '(FP:어 )리터\n',
 '에이치피(휴렛 팩커드)\n',
 '(SN:) 전 트와일라잇이 그렇게 재미있는지 모르겠어요\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n',
 '에이치피(휴렛 팩커드)\n']

In [12]:
df = pd.DataFrame({'path' : path_list, 'label' : label_list})

In [13]:
weird_idx = df[df['label'].apply(lambda x : True if re.compile('\(휴렛 팩커드\)').search(x)!=None else False)].index
df.drop(index = weird_idx, inplace=True)

In [14]:
# 특수 기호가 포함된 샘플은 일단 제거 후 시도해본다
df[df['label'].apply(lambda x : True if re.compile('\(\w.*\)').search(x)!=None else False)]

,path,label
76388,/Data/etc/Robust_ASR/data/hacka/002.노인남녀(시니어)/...,술은 술은 안 먹어야지 (SP:그르) 그나마 쬐금 도움이 되는 거 같애\n
76479,/Data/etc/Robust_ASR/data/hacka/002.노인남녀(시니어)/...,그렇다고 건강관리 안 했다고 해서 (SP:안 아) 안 아픈 것도 아니고\n
104337,/Data/etc/Robust_ASR/data/hacka/002.노인남녀(시니어)/...,이게 (SP:사제) 사계절이 있는 우리나라가 참 내가 사는 곳이지만\n
115647,/Data/etc/Robust_ASR/data/hacka/002.노인남녀(시니어)/...,내가 너한테 얘기 했었나 우리 아들 며칠 전에 베트남 갔다 (SP:왔가) 왔다고\n
261262,/Data/etc/Robust_ASR/data/hacka/004.외래어/외래어_00...,(FP:아) 런던\n
261266,/Data/etc/Robust_ASR/data/hacka/004.외래어/외래어_00...,(FP:어)레몬\n
261330,/Data/etc/Robust_ASR/data/hacka/004.외래어/외래어_00...,(FP:어)룸메이트\n
261338,/Data/etc/Robust_ASR/data/hacka/004.외래어/외래어_00...,(FP:어)리더\n
261362,/Data/etc/Robust_ASR/data/hacka/004.외래어/외래어_00...,(FP:어 )리터\n
270618,/Data/etc/Robust_ASR/data/hacka/004.외래어/외래어_00...,(SN:) 전 트와일라잇이 그렇게 재미있는지 모르겠어요\n


In [15]:
weird_idx_all = df[df['label'].apply(lambda x : True if re.compile('\(\w.*\)').search(x)!=None else False)].index
df.drop(index = weird_idx_all, inplace=True)

In [169]:
# df.to_csv('data_df.csv', index=False)

In [ ]:
df = pd.read_csv('data_df.csv')
df.head()

# PCM -> WAV 변환

In [ ]:
# df['path'].apply(lambda x : pcm2wav(x, x.replace('PCM', 'wav')) if os.path.exists(x) else print(x))

In [17]:
non_exists={}
for idx, row in tqdm(df.iterrows()):
    pcm_path = row['path']
    if os.path.exists(pcm_path):
        pcm2wav(pcm_path, pcm_path.replace('PCM', 'wav'))
    else:
        non_exists[idx] = pcm_path
        df.drop(index = idx, inplace=True)

319915it [37:55, 140.56it/s] 


In [18]:
df[df['path'].apply(lambda x : True if re.compile('성인남녀_002_C_030_F_OSS00_44_충청_녹음실.*').search(x) else False)]['path']

Series([], Name: path, dtype: object)

In [19]:
df.to_csv('dataset_df.csv', index=False)

# age 변수추가
- 성인남녀 : 0
- 시니어 : 1
- 어린이 : 2

In [20]:
re.compile('00[123]\..*').match(df.iloc[-1]['path'].split('/')[-3])

In [21]:
df_age = df[df['path'].apply(lambda x : bool(re.compile('00[123]\..*').match(x.split('/')[-3])))] # 004. 외래어 제외

In [22]:
df_age['age'] = df_age['path'].apply(lambda x : x.split('/')[-3].split('.')[1]) # 한글 레이블
# df_age['age'] = df_age['path'].apply(lambda x : int(x.split('/')[-3].split('.')[0])-1) # 아예 숫자 레이블로

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_age.to_csv('dataset_with_age.csv', index=False)

- train, val 나누기

In [8]:
df_age = pd.read_csv('dataset_with_age.csv')
df_age.head()

,path,label,age
0,/Data/etc/Robust_ASR/data/hacka/001.일반남녀/성인남녀_...,이 대리 아니야\n,일반남녀
1,/Data/etc/Robust_ASR/data/hacka/001.일반남녀/성인남녀_...,안녕하세요 일찍 출근하시네요\n,일반남녀
2,/Data/etc/Robust_ASR/data/hacka/001.일반남녀/성인남녀_...,만원 지하철을 타면 하루 종일 피곤해서 일을 못하겠더라구\n,일반남녀
3,/Data/etc/Robust_ASR/data/hacka/001.일반남녀/성인남녀_...,저도요\n,일반남녀
4,/Data/etc/Robust_ASR/data/hacka/001.일반남녀/성인남녀_...,그래서 아침밥은 포기하고 삼십 분 일찍 출근하기로 했어요\n,일반남녀


In [9]:
df_age.reset_index(drop=True, inplace=True)

In [10]:
df_age['label'] = df_age['label'].apply(lambda x : x.replace('\n', '').replace(',', ''))

In [11]:
random_idx = np.random.choice(df_age.index, size = int(len(df_age) * 0.8), replace=False)
train_index = np.zeros(len(df_age), dtype=bool)
train_index[random_idx] = True

In [12]:
df_age_train = df_age[train_index]
df_age_val = df_age[~train_index]

In [13]:
df_age_train['path'] = df_age_train['path'].apply(lambda x : x.replace('PCM','wav'))
df_age_val['path'] = df_age_val['path'].apply(lambda x : x.replace('PCM','wav'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
# df_age_train.to_csv('./data/csvs/dataset_with_age_train.csv', index=False, header=False)
# df_age_val.to_csv('./data/csvs/dataset_with_age_val.csv', index=False, header=False)

In [19]:
df_age_train = pd.read_csv('./data/csvs/dataset_with_age_train.csv', names = ['path','label','age'])
df_age_val = pd.read_csv('./data/csvs/dataset_with_age_val.csv', names = ['path','label','age'])

In [ ]:
## df_age_train[df_age_train['label'].apply(lambda x : True if re.compile('화면이 한꺼번에.*').search(x) else False )]

In [374]:
df_age_train['path'].loc[79020]

'/Data/etc/Robust_ASR/data/hacka/002.노인남녀(시니어)/시니어_007_C_011_F_HJS00_64_수도권_녹음실/시니어_007_C_011_F_HJS00_64_수도권_녹음실_07298.wav'

In [41]:
df_age_train['length'] = df_age_train['label'].apply(lambda x : len(x)) #length 알아보기
df_age_val['length'] = df_age_val['label'].apply(lambda x : len(x)) #length 알아보기

In [380]:
pcm2wav(df_age_train['path'].loc[85242].replace('wav','PCM'), 'old2.wav')

# 2. ai hub data

In [2]:
aihub_path = '/Data/etc/Robust_ASR/data/aihub'
os.listdir(aihub_path)

['KsponSpeech_01.zip',
 'KsponSpeech_02.zip',
 'KsponSpeech_03.zip',
 'KsponSpeech_04.zip',
 'KsponSpeech_05.zip',
 'KsponSpeech_eval.zip',
 'KsponSpeech_scripts.zip',
 'KsponSpeech_01',
 'KsponSpeech_02',
 'KsponSpeech_03',
 'KsponSpeech_04',
 'KsponSpeech_05',
 'dev.trn',
 'eval_clean.trn',
 'eval_other.trn',
 'train.trn',
 'eval_clean',
 'eval_other',
 '.train.trn.swp']

In [50]:
def aihub_df_pp(label_file):
    path_list = []
    ts_list = []
    with open(os.path.join(aihub_path, label_file), 'r') as f:
        while True:
            row = f.readline()
            if not row:
                break
            path, ts = row.split('::')
            path = os.path.join(aihub_path, path.strip())
            ts = ts.strip().replace(',', '').replace('.','')  # 쉼표 마침표 제거
            ts = ts.replace('b/', '' ).replace('l/', '').replace('n/', '').replace('o/','') #한숨, 웃음, 노이즈, 다른사람 말소리 제거
            path_list.append(path)
            ts_list.append(ts)

    aihub_df = pd.DataFrame({'path' : path_list, 'label' : ts_list})

    # delete unrecognizable sample
    unrecog_idx = aihub_df[aihub_df['label'].apply(lambda x : True if re.compile('u/').search(x) else False)].index
    aihub_df = aihub_df.drop(index=unrecog_idx)

    # 이중전사를 발음전사로 변경!
    aihub_df[aihub_df['label'].apply(lambda x : True if re.compile('\(.*\)/\(.*\)').search(x) else False)] # 눈으료 확인용
    aihub_df = aihub_df.replace('\(\d*.*\)/', '',regex=True)
    aihub_df = aihub_df.replace('\(','', regex=True).replace('\)', '', regex=True)

    # * 샘플처리
    star_idx = aihub_df[aihub_df['label'].apply(lambda x : True if re.compile('\*').search(x) else False)].index
    aihub_df = aihub_df.drop(index = star_idx)

    # + 샘플처리 ->지운다
    plus_idx = aihub_df[aihub_df['label'].apply(lambda x : True if re.compile('\+').search(x) else False)].index
    aihub_df = aihub_df.drop(index = plus_idx)
    
    # 간투어 제거
    aihub_df = aihub_df.replace('[가-핳]/', '', regex=True)
    aihub_df = aihub_df.replace('\?', '', regex=True)  # 물음표 제거

    return aihub_df

In [51]:
aihub_train = aihub_df_pp('train.trn')
aihub_dev = aihub_df_pp('dev.trn')
aihub_eval_clean = aihub_df_pp('eval_clean.trn')
aihub_eval_other = aihub_df_pp('eval_other.trn')

In [67]:
aihub = pd.concat([aihub_train, aihub_dev, aihub_eval_clean, aihub_eval_other], axis=0, ignore_index=True)

In [68]:
aihub['label'] = aihub['label'].apply(lambda x: x.strip())
aihub['length'] = aihub['label'].apply(lambda x : len(x))

In [70]:
# garbage delete
garbage_idx = aihub[aihub['label'].apply(lambda x : True if re.compile('[^가-힣\s]').search(x) else False)] .index
aihub.drop(index=garbage_idx,  inplace=True)

In [77]:
aihub.drop(index= aihub[aihub['length']==0].index, inplace=True) #  length 0 delete
aihub.drop(index = aihub[aihub['length'] > 90].index, inplace=True) # length over 100 delete

In [78]:
aihub.sort_values(by='length', ascending=False)

,path,label,length
470866,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_05...,나 진짜 살면서 클럽 한 번은 가보고 싶었거든 약간 로망이잖아 궁금하잖아 가보고...,90
409453,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_05...,내가 생각하고 있는 전형 이라고 해야 되나 게 아마 내가 알기로는 토플이랑 면접만...,90
129688,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_02...,다른 사람은 나보다 한 이만 원 더 받아 근데 그 사람들이 받는 이유가 거기 팀장...,90
454527,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_05...,맨 앞은 구천 원인가 그러고 중간이 만 원인가 그리고 원래 보통 사람들이 선호하는 ...,90
492696,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_05...,아무튼 같이 뭔가 같이 임무를 맡으면은 걔가 먼저 나서서 해 버려 그래서 내가 할 ...,90
409476,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_05...,그래 가지고 우리 넷이서 가면은 한 삼 인분 시키고 다 먹고 나서 어 뽂음밥 한 세...,90
478095,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_05...,수영장 있구 집 한 채를 다 빌리는 거래 근데 방도 많아서 아 그러면 시간 맞는 ...,90
61142,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_01...,왜 안 취했냐고 몰라 아니 그날 나 안 그래도 감기 걸려서 아 오늘 진짜 컨디...,90
40201,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_01...,많이 별로인 거 같아 우리 같은 경우도 인터넷엔 얘네 왜 이러냐 얜 싫다 이러...,90
260726,/Data/etc/Robust_ASR/data/aihub/KsponSpeech_03...,왜냐면 나도 옛날에 새우 먹고 식중독을 걸린 적이 있어가지고 새우 머리가 안 익었...,90


## PCM to Wav

In [82]:
non_exists={}
for idx, row in tqdm(aihub.iterrows(), total= len(aihub)):
    pcm_path = row['path']
    if os.path.exists(pcm_path):
        pcm2wav(pcm_path, pcm_path.replace('PCM', 'wav'))
    else:
        non_exists[row.name] = pcm_path

100%|██████████| 474650/474650 [49:28<00:00, 159.88it/s]  


In [85]:
# aihub.to_csv('./data/csvs/aihub.csv', index=False)

In [6]:
aihub = pd.read_csv('./data/csvs/aihub_pcm.csv', names=['path','labels','length'])

In [105]:
# aihub['path'] = aihub['path'].apply(lambda x : x.replace('wav', 'pcm'))

In [7]:
# aihub['path'] = aihub['path'].apply(lambda x : x.replace('/Data', '/home/Data'))

In [8]:
# aihub.to_csv('./data/csvs/aihub.csv', index=False, header=False)
# aihub.to_csv('./data/csvs/aihub_pcm.csv', index=False, header=False) # pcm version
# aihub.to_csv('./data/csvs/aihub_pcm_180.csv', index=False, header=False)  #pcm for 180 server

In [35]:
aihub_train.loc[22537,'label']

'인가 상대방이 짝은 마음만 줘도 만족을 할 수 있다는 건 되게 좋은 일인데 그만큼 나도 상대방한테 연애를 하면서 줄 수 있는 마음이 그 정도 인 거니까  만약 상대방이  뭔가 연애를 하면서 주고받을 수 있는 마음이 크기가 훨씬 더 큰 사람이면  뭔가 너한테 그만큼 많이 줄 수 있겠지만 그만큼 반대로 너가 주는 만큼은 나름 너한텐 최선일진 몰라도 그 사람한텐 턱없이 부족할 수 있는 거잖아  그런 데서부터 마 문제가 생기는 거 같애  근데 이제 막 현덕이랑 너 같은 타입은 약간 그런 연애에서 뭔가 약간  그런 바라는 거나 주고 줄 수 있는 게 제일 적은 그런 거인 거고'

In [79]:
pcm2wav(aihub['path'][470866] , 'test.wav')

# combine dataset
- aihub, hacka 데이터합치기

In [3]:
hacka_train = pd.read_csv('./data/csvs/dataset_with_age_train.csv', names = ['path','label','age'])
hacka_val = pd.read_csv('./data/csvs/dataset_with_age_val.csv', names = ['path','label','age'])
aihub = pd.read_csv('./data/csvs/aihub_manifest.csv', names = ['path','label','age'])

In [7]:
hacka_train['path'] = hacka_train['path'].apply(lambda x: x.replace('/Data', '/home/Data'))
hacka_val['path'] = hacka_val['path'].apply(lambda x: x.replace('/Data', '/home/Data'))

In [11]:
combine = pd.concat([hacka_train, hacka_val, aihub], axis=0, ignore_index=True)

In [15]:
combine.to_csv('./data/csvs/combine_180.csv', index=False, header=False)

In [16]:
random_idx = np.random.choice(combine.index, size = int(len(combine) * 0.8), replace=False)
train_index = np.zeros(len(combine), dtype=bool)
train_index[random_idx] = True

In [17]:
combine_train = combine[train_index]
combine_val = combine[~train_index]

In [18]:
combine_train.to_csv('./data/csvs/combine_train_180.csv', index=False, header=False)
combine_val.to_csv('./data/csvs/combine_test_180.csv', index=False, header=False)